<a href="https://colab.research.google.com/github/JazmineOrtizMarin/Simulaci-n-2/blob/main/Proyecto_MW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import heapq
import random
from dataclasses import dataclass, field
from enum import Enum
from typing import List, Optional

In [1]:
# Definiciones de Estados y Tipos
class EstadoPaciente(Enum):
    EN_COLA = 1
    EN_SERVICIO = 2
    ATENDIDO = 3
    ABANDONO = 4

class TipoEvento(Enum):
    LLEGADA = 1
    INICIO_SERVICIO = 2 # a veces se maneja directo en la lógica
    FIN_SERVICIO = 3
    ABANDONO = 4

# Clases de Datos (Entidades y Eventos)

@dataclass
class Paciente:
    id: int
    tiempo_llegada: float
    tiempo_limite_espera: float # Tiempo absoluto en el que se irá (llegada + paciencia)
    estado: EstadoPaciente = EstadoPaciente.EN_COLA
    tiempo_inicio_servicio: Optional[float] = None
    tiempo_salida: Optional[float] = None

@dataclass
class Doctor:
    id: int
    ocupado: bool = False
    paciente_actual: Optional[Paciente] = None

@dataclass(order=True)
class Evento:
    tiempo: float
    tipo: TipoEvento = field(compare=False) # No comparar por tipo, solo por tiempo
    paciente: Optional[Paciente] = field(compare=False, default=None)
    doctor: Optional[Doctor] = field(compare=False, default=None)

    # El decorador @dataclass(order=True) genera automáticamente el __lt__
    # basándose en el primer atributo ('tiempo'), que es lo que heapq necesita.

# Motor de Simulación

class Clinica:
    def __init__(self, env_lambda, env_mu, c_doctores, tiempo_max_simulacion):
        # Parámetros
        self.lambd = env_lambda
        self.mu = env_mu
        self.c = c_doctores
        self.tiempo_max = tiempo_max_simulacion

        # Estado del sistema
        self.reloj = 0.0
        self.cola_eventos = [] # Nuestra Priority Queue
        self.doctores = [Doctor(i) for i in range(c_doctores)]
        self.cola_pacientes = [] # Deque o lista

        # Estadísticas y Contadores
        self.pacientes_totales = 0
        self.pacientes_atendidos = 0
        self.pacientes_abandonaron = 0
        # Listas para guardar métricas (tiempos de espera, etc.)
        self.tiempos_espera_cola = []

    def agendar_evento(self, evento: Evento):
        heapq.heappush(self.cola_eventos, evento)

    def correr(self):
        # Primer evento: Llega el primer paciente
        self.agendar_evento(Evento(0.0, TipoEvento.LLEGADA))

        while self.cola_eventos and self.reloj < self.tiempo_max:
            evento_actual = heapq.heappop(self.cola_eventos)
            self.reloj = evento_actual.tiempo

            self.procesar_evento(evento_actual)

    def procesar_evento(self, evento):
        if evento.tipo == TipoEvento.LLEGADA:
            self.manejar_llegada(evento)
        elif evento.tipo == TipoEvento.ABANDONO:
            self.manejar_abandono(evento)
        elif evento.tipo == TipoEvento.FIN_SERVICIO:
            self.manejar_fin_servicio(evento)

    # Lógica Específica

    def manejar_llegada(self, evento):
        # 1. Crear paciente
        self.pacientes_totales += 1
        # Generar paciencia aleatoria (ej. exponencial o uniforme)
        paciencia = random.uniform(10, 60) # Ejemplo
        tiempo_abandono = self.reloj + paciencia

        nuevo_paciente = Paciente(
            id=self.pacientes_totales,
            tiempo_llegada=self.reloj,
            tiempo_limite_espera=tiempo_abandono
        )

        # 2. Agendar su posible abandono FUTURO
        ev_abandono = Evento(tiempo_abandono, TipoEvento.ABANDONO, paciente=nuevo_paciente)
        self.agendar_evento(ev_abandono)

        # 3. Intentar asignar doctor o mandar a cola
        doctor_libre = self.buscar_doctor_libre()
        if doctor_libre:
            self.iniciar_servicio(nuevo_paciente, doctor_libre)
        else:
            self.cola_pacientes.append(nuevo_paciente)

        # 4. Agendar la SIGUIENTE llegada (Bootstrapping)
        prox_llegada = self.reloj + random.expovariate(self.lambd)
        self.agendar_evento(Evento(prox_llegada, TipoEvento.LLEGADA))

    def manejar_abandono(self, evento):
        paciente = evento.paciente
        # Verificar si el paciente SIGUE en la cola.
        # Si ya está siendo atendido o ya se fue, ignoramos este evento.
        if paciente.estado == EstadoPaciente.EN_COLA:
            # Lo sacamos de la cola (esto es O(n) en listas, cuidado si la cola es gigante)
            if paciente in self.cola_pacientes:
                self.cola_pacientes.remove(paciente)
                paciente.estado = EstadoPaciente.ABANDONO
                paciente.tiempo_salida = self.reloj
                self.pacientes_abandonaron += 1
                # Registrar métricas de abandono aquí

    def buscar_doctor_libre(self):
        for doc in self.doctores:
            if not doc.ocupado:
                return doc
        return None

    def iniciar_servicio(self, paciente, doctor):
        paciente.estado = EstadoPaciente.EN_SERVICIO
        paciente.tiempo_inicio_servicio = self.reloj
        doctor.ocupado = True
        doctor.paciente_actual = paciente

        # Agendar fin de servicio
        duracion = random.expovariate(self.mu)
        self.agendar_evento(Evento(self.reloj + duracion, TipoEvento.FIN_SERVICIO, paciente, doctor))

    def manejar_fin_servicio(self, evento):
        doctor = evento.doctor
        paciente = evento.paciente

        # Cerrar datos del paciente
        paciente.estado = EstadoPaciente.ATENDIDO
        paciente.tiempo_salida = self.reloj
        self.pacientes_atendidos += 1
        doctor.ocupado = False
        doctor.paciente_actual = None

        # IMPORTANTE: Si hay alguien en cola, atenderlo INMEDIATAMENTE
        if self.cola_pacientes:
            siguiente_paciente = self.cola_pacientes.pop(0) # FIFO
            self.iniciar_servicio(siguiente_paciente, doctor)